In [0]:
"""
Created on Tue Oct 15 00:14:08 2019

@author: anil
"""

import pandas as pd
import numpy as np
     
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 

import nltk
nltk.download("popular")
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
import csv
from google.colab import drive
drive.mount('/content/drive')



[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

## Data Reading

In [0]:
"""
c"""

train=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/TRAIN-TURK-ENCODED')
test=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/TEST-TURK-ENCODED')

X_train_FULL=train[['id','tweet_initial_nontoken','tweet','tweet_initial','subtask_a']]
Y_TRAIN_ENCODED_FULL=train['subtask_a']
X_test_FULL=test[['id','tweet_initial_nontoken','tweet','tweet_initial','subtask_a']]
Y_TEST_ENCODED_FULL=test['subtask_a']

In [0]:
"""
TEST MODE WİTH DOWNSAMPLING
"""

train=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/TRAIN-TURK-ENCODED')
test=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/TEST-TURK-ENCODED')

X_train=train[['id','tweet_initial_nontoken','tweet','tweet_initial','subtask_a']]
y_train_encoded=train['subtask_a']
X_test=test[['id','tweet_initial_nontoken','tweet','tweet_initial','subtask_a']]
y_test_encoded=test['subtask_a']

import csv
X_train_FULL=pd.concat([X_train,X_test])
X_test_FULL=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/offenseval-tr-testset-v1.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TRAIN_ENCODED_FULL=pd.concat([y_train_encoded,y_test_encoded])
Y_TEST=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/turkish-goldlabels.tsv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]


In [0]:
Y_TEST_ENCODED_FULL

In [0]:

"""
Test mode Without downsampling

"""
olid_training=pd.read_csv("/content/drive/My Drive/OFFENSEVAL20-DATA/offenseval-tr-training-v1.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
X_test_FULL=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/offenseval-tr-testset-v1.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/turkish-goldlabels.tsv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

In [0]:
"""VALID MODE without downsampling
"""
from nltk.tokenize import word_tokenize 
import nltk
olid_training=pd.read_csv("/content/drive/My Drive/OFFENSEVAL20-DATA/offenseval-tr-training-v1.tsv",sep="\t")
from sklearn.model_selection import train_test_split
train_olid, test_olid = train_test_split(olid_training, test_size=0.20,random_state=11)

X_train_FULL=train_olid[["id","tweet","subtask_a"]] 
Y_train_FULL=train_olid["subtask_a"]
X_test_FULL=test_olid[["id","tweet","subtask_a"]]
Y_test_FULL=np.array(test_olid[["subtask_a"]])

Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_test_FULL]


## Pre-processing

In [0]:
"""
	 Converting all text to lowercase 
	 Removing all "\#" symbols
	 Removing all punctuation
	 Removing @user tokens
"""
X_train_FULL.tweet = X_train_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_train_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_train_FULL["tweet_initial"]=filtered_tweets


X_test_FULL.tweet = X_test_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_test_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_test_FULL["tweet_initial"]=filtered_tweets




#FOR TEST

z=[]
for tweet in X_train_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_train_FULL["tweet_initial_nontoken"]=z



#FOR TEST

z=[]
for tweet in X_test_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test_FULL["tweet_initial_nontoken"]=z



In [0]:
"""
Importing Keras dependencies / Write Custom Evaluation Metrices
"""

from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from tensorflow.keras.layers import Embedding
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
def recall_m(true_Y, pred_Y):
        TP = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        possible_pos = K.sum(K.round(K.clip(true_Y, 0, 1)))
        rec = TP / (possible_pos + K.epsilon())
        return rec

def precision_m(true_Y, pred_Y):
        true_positives = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(pred_Y, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


    

def f1_m(true_Y, pred_Y):
    pres = precision_m(true_Y, pred_Y)
    rec = recall_m(true_Y, pred_Y)
    return 2*((pres*rec)/(pres+rec+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


## Tweeter Word2vec /

In [0]:
fname= "/content/drive/My Drive/Twitter/Word2Vec/w2v_model_word.vec"
import gensim
from gensim.models import Word2Vec
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname)  # you can load this saved keyedvectors model later



## Public Word2vec


In [0]:

#fname='/content/drive/My Drive/Public-Embeddings/word2vec/haber-P1_S1_L0-100V_5E_1547557988.bin.wv.vectors.npy'
fname= "/content/drive/My Drive/Public-Embeddings/word2vec/haber-P1_S1_L0-100V_5E_1547557988.bin"

#fname= '/content/drive/My Drive/OFFENSEVAL20-DATA/haber-P0_S1_L0-100V_5E_v2v.bin'
import gensim
from gensim.models import Word2Vec



#word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname,encoding='utf8',binary=True)  # you can load this saved keyedvectors model later

#word_vectors=gensim.models.Word2Vec.load(fname)

word_vectors=gensim.models.KeyedVectors.load(fname)
#models = gensim.models.Word2Vec.load(fname)

#gensim.models.KeyedVectors.load_word2vec_format(fname,encoding='utf8')


UnpicklingError: ignored

## Tweeter Fasttext


In [0]:
import gensim
from gensim.models import FastText

word_vectors = gensim.models.FastText.load_fasttext_format('/content/drive/My Drive/Twitter/FastText/fastText_25022020.bin',encoding='utf-8') # use that if you want to use fasttedxt 



## Public FastText

In [0]:
word_vectors = gensim.models.FastText.load_fasttext_format('/content/drive/My Drive/OFFENSEVAL20-DATA/haber-P1_S0_L0.bin',encoding='utf-8') # use that if you want to use fasttedxt 


## Tokenizing / creating vocabulary and wordindex using keras functinalities


In [0]:
"""
We will use word indexes as look-up table during embedding layer.
"""
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=98790)  #the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
tokenizer.fit_on_texts(X_train_FULL["tweet_initial_nontoken"])
X_train_initial = tokenizer.texts_to_sequences(X_train_FULL["tweet_initial_nontoken"])
X_test_initial = tokenizer.texts_to_sequences(X_test_FULL["tweet_initial_nontoken"])
vocab_size_initial = len(tokenizer.word_index) + 1 
wordIndex_initial=tokenizer.word_index # it is  index
from keras.preprocessing.sequence import pad_sequences
max_len = 50

"""
Padding

"""
X_train_initial = pad_sequences(X_train_initial, padding='post', maxlen=max_len)
X_test_initial = pad_sequences(X_test_initial, padding='post', maxlen=max_len)




Custom Embedding Layer

In [0]:
 """
Custom Embedding Layer with Word2vec weights
 """

def createEmbeddingLayer(wordIndex,not_static):
  a=[]
  embedding_dim=300
  vocabulary_size=len(wordIndex)+1
  embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
  missed=0
  for word, i in wordIndex.items():
    
          
      try:
          embedding_vector = word_vectors[word] # or fast text
          embedding_matrix[i] = embedding_vector
            

      except KeyError: # If word is not found in the word2vec vocabulary , assign random weights
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)
        missed+=1
        a.append(word)

  print('missed_words :' , missed)

  custom_embedding_layer = Embedding(vocabulary_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                trainable=not_static )# Controls the updating weights )
  return custom_embedding_layer

  

## BiLSTM

In [0]:
"""
BiLSTM-Attention Network

Benefited from https://github.com/ShawnyXiao/TextClassification-Keras#5-textattbirnn

Implementation of https://arxiv.org/abs/1512.08756
"""


from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional


In [0]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, Dense, Bidirectional,LSTM

def sigmoid(x): 
    return 1.0/(1 + np.exp(-x))


class biLSTM(object):
    def __init__(self, maxlen, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))
        embedding = createEmbeddingLayer(wordIndex_initial,False)(input) #change
        x = Bidirectional(LSTM(128, return_sequences=False))(embedding)  # LSTM or GRU
        x = Dense(128, activation='relu')(x)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

#Benefited from https://github.com/ShawnyXiao/TextClassification-Keras#5-textattbirnn
 

In [0]:
def train(X_train_initial,X_test_initial,y_train,y_test) : 
  LSTM_MODEL= biLSTM(50, 100).get_model()

  early_stopping = [EarlyStopping(monitor='val_loss',min_delta=0,restore_best_weights=True, patience=10,verbose=1, mode='auto')]
  LSTM_MODEL.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc',f1_m,precision_m, recall_m])
  LSTM_MODEL.summary()

  LSTM_MODEL.fit(X_train_initial, y_train,
            batch_size=16,
            epochs=30,
            callbacks=early_stopping,
            validation_split=0.1)

  loss, accuracy, f1_score, precision, recall = LSTM_MODEL.evaluate(X_train_initial, y_train, verbose=1)
  print(" biLSTM Training Loss: {:.4f}".format(loss))
  print(" biLSTM Accuracy: {:.4f}".format(accuracy))
  print("  biLSTM  f1 score: {:.4f}".format(f1_score))
  print("Attention- BiRNN Precision: {:.4f}".format(precision))
  print("Attention- BiRNN Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = LSTM_MODEL.evaluate(X_test_initial, y_test, verbose=1)
  print(" biLSTM Test Loss: {:.4f}".format(loss))
  print(" biLSTM Test Accuracy: {:.4f}".format(accuracy))
  print("  biLSTM Test f1 score: {:.4f}".format(f1_score))
  print(" biLSTM  Test Precision: {:.4f}".format(precision))
  print(" biLSTM Test Recall: {:.4f}".format(recall))
 
  probs = LSTM_MODEL.predict(X_test_initial, verbose=1)
  print('lenght of probs : ' ,len(probs))
  predicted_classes=[0 if i < 0.5 else 1 for i in probs]
 

  print(classification_report(y_test, predicted_classes,digits=3))
  return (predicted_classes,probs)





In [0]:
"""
 test  MODE WİTHOUT DOWNSAMPLING
 "tweeter word2vec" 
"""

prediction_attention,probs_attention=train(np.array(X_train_initial),np.array(X_test_initial),np.array(Y_TRAIN_ENCODED_FULL),np.array(Y_TEST_ENCODED_FULL))

missed_words : 23841
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 50, 300)           29637000  
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
Total params: 30,109,321
Trainable params: 472,321
Non-trainable params: 29,637,000
_________________________________________________________________
Epoch 1/30
1760/1760 [==============================] - 16s 9ms/step - loss: 0.3476 - 

In [0]:
"""
 VALID  MODE WİTHOUT DOWNSAMPLING
 "tweeter word2vec" 
"""

prediction_attention,probs_attention=train(np.array(X_train_initial),np.array(X_test_initial),np.array(Y_TRAIN_ENCODED_FULL),np.array(Y_TEST_ENCODED_FULL))

missed_words : 18908
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 50, 300)           25218300  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 25,690,621
Trainable params: 472,321
Non-trainable params: 25,218,300
_________________________________________________________________
Epoch 1/30
1408/1408 [==============================] - 13s 9ms/step - loss: 0.3538 - 

In [0]:
"""
 TEST MODE WİTH DOWNSAMPLING
 "tweeter word2vec" 
"""

prediction_attention,probs_attention=train(np.array(X_train_initial),np.array(X_test_initial),np.array(Y_TRAIN_ENCODED_FULL),np.array(Y_TEST_ENCODED_FULL))

missed_words : 17612
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 50, 300)           24060600  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 24,532,921
Trainable params: 472,321
Non-trainable params: 24,060,600
_________________________________________________________________
Epoch 1/30
1300/1300 [==============================] - 12s 9ms/step - loss: 0.4118 - 

In [0]:
"""
 VALID MODE WİTH DOWNSAMPLING
 "tweeter word2vec" 
"""

prediction_attention,probs_attention=train(np.array(X_train_initial),np.array(X_test_initial),np.array(Y_TRAIN_ENCODED_FULL),np.array(Y_TEST_ENCODED_FULL))

missed_words : 13202
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 300)           19428000  
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               439296    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 19,900,321
Trainable params: 472,321
Non-trainable params: 19,428,000
_________________________________________________________________
Epoch 1/30
948/948 [==============================] - 9s 10ms/step - loss: 0.4299 - acc:

In [0]:
bilstmatt_test_full_result= pd.DataFrame()
bilstmatt_test_full_result['biLSTM-ATTENTION-PREDICTON']=prediction_attention
bilstmatt_test_full_result['biLSTM-ATTENTION-PROBS']=probs_attention
bilstmatt_test_full_result['Gold-label']=Y_TEST_ENCODED_FULL

In [0]:
bilstmatt_test_full_result

,biLSTM-ATTENTION-PREDICTON,biLSTM-ATTENTION-PROBS,Gold-label
0,0,0.022655,0
1,0,0.125425,0
2,0,0.069387,0
3,0,0.402815,1
4,1,0.570750,1
...,...,...,...
3523,0,0.018026,0
3524,0,0.006181,0
3525,0,0.014974,0
3526,0,0.037331,0


In [0]:
bilstmatt_test_full_result.to_csv('bilstmatt_test_full_result.csv',header=False)

In [0]:
bilstmatt_valid_full_result= pd.DataFrame()
bilstmatt_valid_full_result['biLSTM-ATTENTION-PREDICTON']=prediction_attention
bilstmatt_valid_full_result['biLSTM-ATTENTION-PROBS']=probs_attention
bilstmatt_valid_full_result['Gold-label']=Y_TEST_ENCODED_FULL
bilstmatt_valid_full_result.to_csv('bilstmatt_valid_full_result.csv',header=False)

In [0]:
bilstmatt_valid_full_result

,biLSTM-ATTENTION-PREDICTON,biLSTM-ATTENTION-PROBS,Gold-label
0,0,0.082957,0
1,0,0.070353,0
2,0,0.036576,0
3,0,0.055747,0
4,0,0.031174,0
...,...,...,...
6251,0,0.045234,0
6252,0,0.309828,0
6253,0,0.022087,0
6254,0,0.123876,1
